##  Let me try [Parrelle is all you want](https://github.com/IliaZenkov/transformer-cnn-emotion-recognition/blob/main/notebooks/Parallel_is_All_You_Want.ipynb), but only adaopting its model.
 20 Oct 2020

In [7]:
!git clone https://github.com/fastai/course-v3/ /content/course-v3
%cd /content/course-v3/nbs/dl2

Cloning into '/content/course-v3'...
remote: Enumerating objects: 5893, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5893 (delta 0), reused 2 (delta 0), pack-reused 5890
Receiving objects: 100% (5893/5893), 263.03 MiB | 33.21 MiB/s, done.
Resolving deltas: 100% (3249/3249), done.
/content/course-v3/nbs/dl2


- NOTE: if you do not have installed [nvidia/apex](https://github.com/nvidia/apex),  change `from exp.nb_10c import *` to `from exp.nb_10b import *` in `exp.nb_11`

In [9]:
from exp.nb_12a import *

- Local data (which is divided into separate dataset) is saved to `Path('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/v1/ser'`


In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [11]:
root_path = Path('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1'); root_path.ls()

[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/dev')]

features-entry contains a list of a list with 26 items.
- length of inner list: 26 (float numbers - represent one preprocessed speech frame (logMel))

- length of outer list: number of frames per data-point, e.g. 10 or 15, ...

In [12]:
train_path = root_path/'train'

In [13]:
audios = get_files(train_path)

In [14]:
# restart-run
class AudioList(ItemList):
    @classmethod
    def from_files(cls, path, extensions = None, recurse=True, include=None, **kwargs):
        return cls(get_files(path, extensions, recurse=recurse, include=include), path, **kwargs)
    
    def get(self, fn):
        return torch.load(fn)

In [15]:
al=AudioList.from_files(train_path); al

AudioList (7800 items)
[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7760_1_1.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7761_0_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7762_0_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7763_1_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7764_1_1.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7765_1_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7766_1_1.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7767_1_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7768_1_0.p

## Use adaptive pooling to fit the frames

## input transform

1. load tensor
2. tocuda
3. transpose, add dimension to first axis
4. AdaptiveAvgPooling to fit width(time frames) 250


In [16]:
class ToCuda():
    _order=10
    def __call__(self, ad):
        return ad.cuda()

In [17]:
class Reshape():
    _order=11
    def __call__(self, item):
        w, h = item.shape
        return item.view(h, w)

In [18]:
class DummyFeature():
    # add features from function
    _order= 13
    def __init__(self, f, **kwargs):
        self.p = partial(f, **kwargs)
    def __call__(self, item):
        return torch.vstack((item, self.p(item).unsqueeze(0)))

In [19]:
# Mutants of input tensor
class PadorTrim():
    _order = 20
    def __init__(self, max_len):
        self.max_len = max_len
    def __call__(self, ad):
        # h - logmel, here 27, w - frames / various
        h, w = ad.shape
        pad_size = self.max_len - w
        if pad_size >0: return torch.cat((ad, torch.zeros(h, pad_size).cuda()), dim=1)
        else: return ad[:, :self.max_len]

class TimeFramePooling():
    _order= 12
    def __init__(self, f, n_tf):
        # n_tf: numner of time frames
        self.p = f(n_tf)
    def __call__(self, item):
        return self.p(item)

# TimeFramePooling(f= nn.AdaptiveMaxPool1d, n_tf = 150)

class DummyChannel():
    _order = 30
    def __call__(self, item):
        return item.unsqueeze(0)

In [20]:
tfms = [ToCuda(), Reshape(), DummyFeature(torch.mean, dim=0), PadorTrim(300), DummyChannel()]
compose(al[0], tfms).shape

torch.Size([1, 27, 300])

In [21]:
il = AudioList.from_files(train_path, tfms = tfms)

- This scaling process is annoying, because I should have this process 'before' training, while separating train and eval status

In [14]:
il[0].shape

torch.Size([1, 27, 300])

In [22]:
def re_labeler(fn, pat, subcl='act'):
    assert subcl in ['act', 'val', 'all']
    if subcl=='all': return ''.join(re.findall(pat, str(fn)))
    else:
        return re.findall(pat, str(fn))[0] if pat == 'act' else re.findall(pat, str(fn))[1]

label_pat = r'_(\d+)'
emotion_labeler = partial(re_labeler, pat=label_pat, subcl='all')

In [23]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.2))
ll = label_by_func(sd, emotion_labeler, proc_y=CategoryProcessor())

In [24]:
bs=64
c_in = ll.train[0][0].shape[0]
c_out = len(uniqueify(ll.train.y))
data = ll.to_databunch(bs,c_in=c_in,c_out=c_out)

In [149]:
c_in, c_out

(1, 4)

In [150]:
x, y= ll.train[0]

In [151]:
x.shape, y

(torch.Size([1, 27, 300]), 0)

In [152]:
xb, yb = next(iter(data.train_dl))

In [153]:
xb.shape, yb.shape

(torch.Size([64, 1, 27, 300]), torch.Size([64]))

In [25]:
X_train = torch.cat(tuple(data.train_ds.x[i] for i in range(len(data.train_ds))), dim=0).unsqueeze(1).cuda()
X_valid = torch.cat(tuple(data.valid_ds.x[i] for i in range(len(data.valid_ds))), dim=0).unsqueeze(1).cuda()
y_train = torch.tensor(data.train_ds.y.items)
y_valid = torch.tensor(data.valid_ds.y.items)
X_train.shape, X_valid.shape

(torch.Size([6176, 1, 27, 300]), torch.Size([1624, 1, 27, 300]))

## Model

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, glob
import IPython
from IPython.display import Audio
from IPython.display import Image

In [3]:
from pathlib import Path
Path.ls = lambda x: list(x.iterdir())

In [4]:
root_path = Path('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser'); root_path.ls()

[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v0'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v2.1'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/ser-data-v2-1.tar.gz')]

In [5]:
!cp {root_path/'ser-data-v2-1.tar.gz'} .

In [6]:
!tar -xzf ser-data-v2-1.tar.gz

In [7]:
data_path = Path('/content/v2.1')

In [8]:
def get_data(fpath):
    return [torch.load(i) for i in fpath.iterdir() if i.suffix=='.pt']

In [34]:
X_train, y_train, y_valid, X_valid = get_data(data_path)

In [36]:
list(map(lambda x: print(f"{x.shape}"), (X_train, y_train, X_valid, y_valid)))

torch.Size([6176, 1, 27, 300])
torch.Size([6176])
torch.Size([1624, 1, 27, 300])
torch.Size([1624])


[None, None, None, None]

In [10]:
X_train.shape, X_train.type(), X_train.device

(torch.Size([6176, 1, 27, 300]), 'torch.FloatTensor', device(type='cpu'))

In [ ]:
#@title
model_demo = nn.Sequential(
    
 nn.Sequential(   nn.Conv2d(
        in_channels=1, # input volume depth == input channel dim == 1
        out_channels=16, # expand output feature map volume's depth to 16
        kernel_size=3, # typical 3*3 stride 1 kernel
        stride=1,
        padding=1
                ),
    nn.BatchNorm2d(16), # batch normalize the output feature map before activation
    nn.ReLU(), # feature map --> activation map
    nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
    nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
    ),
    # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
nn.Sequential(    nn.Conv2d(
        in_channels=16, 
        out_channels=32, # expand output feature map volume's depth to 32
        kernel_size=3,
        stride=1,
        padding=1
                ),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4), # increase maxpool kernel for subsequent filters
    nn.Dropout(p=0.3)),
    
    # 3rd 2D convolution layer identical to last except output dim
nn.Sequential(    nn.Conv2d(
        in_channels=32,
        out_channels=64, # expand output feature map volume's depth to 64
        kernel_size=3,
        stride=1,
        padding=1
                ),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.AdaptiveMaxPool2d((1,4)),
    nn.Dropout(p=0.3),)
    )

In [37]:
class parallel_all_you_want(nn.Module):
    # Define all layers present in the network
    def __init__(self,num_emotions):
        super().__init__() 
        self.transformer_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=27, # input feature (frequency) dim after maxpooling 26*250 -> 26*62 (MFC*time)
            nhead=3, # 4 self-attention layers in each multi-head self-attention layer in each encoder block
            dim_feedforward=256, # 2 linear layers in each encoder block's feedforward network: dim 26-->512--->26
            dropout=0.4, 
            activation='gelu' # ReLU: avoid saturation/tame gradient/reduce compute time
        )
        self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=3)
        self.conv2Dblock1 = nn.Sequential(
            
            # 1st 2D convolution layer
            nn.Conv2d(
                in_channels=1, # input volume depth == input channel dim == 1
                out_channels=16, # expand output feature map volume's depth to 16
                kernel_size=3, # typical 3*3 stride 1 kernel
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(16), # batch normalize the output feature map before activation
            nn.ReLU(), # feature map --> activation map
            nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
            nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
            
            # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
            nn.Conv2d(
                in_channels=16, 
                out_channels=32, # expand output feature map volume's depth to 32
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4), # increase maxpool kernel for subsequent filters
            nn.Dropout(p=0.3), 
            
            # 3rd 2D convolution layer identical to last except output dim
            nn.Conv2d(
                in_channels=32,
                out_channels=64, # expand output feature map volume's depth to 64
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d((1,4)),
            nn.Dropout(p=0.3),
        )
        self.conv2Dblock2 = nn.Sequential(
            
            # 1st 2D convolution layer
            nn.Conv2d(
                in_channels=1, # input volume depth == input channel dim == 1
                out_channels=16, # expand output feature map volume's depth to 16
                kernel_size=3, # typical 3*3 stride 1 kernel
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(16), # batch normalize the output feature map before activation
            nn.ReLU(), # feature map --> activation map
            nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
            nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
            
            # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
            nn.Conv2d(
                in_channels=16, 
                out_channels=32, # expand output feature map volume's depth to 32
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4), # increase maxpool kernel for subsequent filters
            nn.Dropout(p=0.3), 
            
            # 3rd 2D convolution layer identical to last except output dim
            nn.Conv2d(
                in_channels=32,
                out_channels=64, # expand output feature map volume's depth to 64
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d((1,4)),
            nn.Dropout(p=0.3),
        )
        self.fc1_linear = nn.Linear(256*2+27, num_emotions)
        self.softmax_out = nn.Softmax(dim=1) # dim==1 is the freq embedding
    def forward(self,x):
        conv2d_embedding1 = self.conv2Dblock1(x)
        conv2d_embedding1 = torch.flatten(conv2d_embedding1, start_dim=1) 
        conv2d_embedding2 = self.conv2Dblock2(x)
        conv2d_embedding2 = torch.flatten(conv2d_embedding2, start_dim=1)
        x_maxpool = self.transformer_maxpool(x)
        x_maxpool_reduced = torch.squeeze(x_maxpool,1)
        x = x_maxpool_reduced.permute(2,0,1) 
        transformer_output = self.transformer_encoder(x)
        transformer_embedding = torch.mean(transformer_output, dim=0) # dim 27x75 --> 27
        complete_embedding = torch.cat([conv2d_embedding1, conv2d_embedding2,transformer_embedding], dim=1)
        output_logits = self.fc1_linear(complete_embedding)
        output_softmax = self.softmax_out(output_logits)
        
        return output_logits, output_softmax

In [12]:
def criterion(predictions, targets): 
    return nn.CrossEntropyLoss()(input=predictions, target=targets)

In [44]:
def make_train_step(model, criterion, optimizer):
    
    # define the training step of the training phase
    def train_step(X,Y):
        
        # forward pass
        output_logits, output_softmax = model(X)
        fscore = f1(output_softmax.argmax(dim=1).cpu(), Y.cpu())
        
        # compute loss on logits because nn.CrossEntropyLoss implements log softmax
        loss = criterion(output_logits, Y) 
        
        # compute gradients for the optimizer to use 
        loss.backward()
        
        # update network parameters based on gradient stored (by calling loss.backward())
        optimizer.step()
        
        # zero out gradients for next pass
        # pytorch accumulates gradients from backwards passes (convenient for RNNs)
        optimizer.zero_grad() 
        
        return loss.item(), fscore
    return train_step

In [45]:
def make_validate_fnc(model,criterion):
    def validate(X,Y):
        
        # don't want to update any network parameters on validation passes: don't need gradient
        # wrap in torch.no_grad to save memory and compute in validation phase: 
        with torch.no_grad(): 
            
            # set model to validation phase i.e. turn off dropout and batchnorm layers 
            model.eval()
      
            # get the model's predictions on the validation set
            # print(X.shape)
            output_logits, output_softmax = model(X)
            fscore = f1(output_softmax.argmax(dim=1).cpu(), Y.cpu())
            
            # compute error from logits (nn.crossentropy implements softmax)
            loss = criterion(output_logits,Y)
            
        return loss.item(), fscore, output_softmax.argmax(dim=1)
    return validate


In [46]:
def make_save_checkpoint(): 
    def save_checkpoint(optimizer, model, epoch, filename):
        checkpoint_dict = {
            'optimizer': optimizer.state_dict(),
            'model': model.state_dict(),
            'epoch': epoch
        }
        torch.save(checkpoint_dict, filename)
    return save_checkpoint

In [47]:
def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch

In [48]:
# get training set size to calculate # iterations and minibatch indices
train_size = X_train.shape[0]

# pick minibatch size (of 32... always)
minibatch = 32

# set device to GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'{device} selected')

# instantiate model and move to GPU for training
model = parallel_all_you_want(num_emotions=4).to(device) 
print('Number of trainable params: ',sum(p.numel() for p in model.parameters()))

# encountered bugs in google colab only, unless I explicitly defined optimizer in this cell...
optimizer = torch.optim.SGD(model.parameters(),lr=0.01, weight_decay=1e-3, momentum=0.8)

# instantiate the checkpoint save function
save_checkpoint = make_save_checkpoint()

# instantiate the training step function 
train_step = make_train_step(model, criterion, optimizer=optimizer)

# instantiate the validation loop function
validate = make_validate_fnc(model,criterion)

# instantiate lists to hold scalar performance metrics to plot later
train_losses=[]
valid_losses = []

cuda selected
Number of trainable params:  100917


In [50]:
model_path = Path(f'/content/models/checkpoints')
model_path.mkdir(parents = True, exist_ok=True)

In [51]:
# create training loop for one complete epoch (entire training set)
def train(optimizer, model, num_epochs, X_train, Y_train, X_valid, Y_valid):

    for epoch in range(num_epochs):
        
        # set model to train phase
        model.train()         
        
        # shuffle entire training set in each epoch to randomize minibatch order
        train_indices = np.random.permutation(train_size) 
        
        # shuffle the training set for each epoch:
        X_train = X_train[train_indices,:,:,:] 
        Y_train = Y_train[train_indices]

        # instantiate scalar values to keep track of progress after each epoch so we can stop training when appropriate 
        epoch_acc = 0 
        epoch_loss = 0
        num_iterations = int(train_size / minibatch)
        
        # create a loop for each minibatch of 32 samples:
        for i in range(num_iterations):
            
            # we have to track and update minibatch position for the current minibatch
            # if we take a random batch position from a set, we almost certainly will skip some of the data in that set
            # track minibatch position based on iteration number:
            batch_start = i * minibatch 
            # ensure we don't go out of the bounds of our training set:
            batch_end = min(batch_start + minibatch, train_size) 
            # ensure we don't have an index error
            actual_batch_size = batch_end-batch_start 
            
            # get training minibatch with all channnels and 2D feature dims
            X = X_train[batch_start:batch_end,:,:,:] 
            # get training minibatch labels 
            Y = Y_train[batch_start:batch_end] 

            # instantiate training tensors
            X_tensor = X.clone().detach().to(device)
            Y_tensor = Y.clone().detach().to(device)

            # Pass input tensors thru 1 training step (fwd+backwards pass)
            loss, acc = train_step(X_tensor, Y_tensor) 
            
            # aggregate batch accuracy to measure progress of entire epoch
            epoch_acc += acc * actual_batch_size / train_size
            epoch_loss += loss * actual_batch_size / train_size
            
            # keep track of the iteration to see if the model's too slow
            print('\r'+f'Epoch {epoch}: iteration {i}/{num_iterations}',end='')
        
        
        X_valid_tensor = X_valid.clone().detach().to(device)
        Y_valid_tensor = Y_valid.clone().detach().to(device)
        
        # calculate validation metrics to keep track of progress; don't need predictions now
        valid_loss, valid_acc, _ = validate(X_valid_tensor, Y_valid_tensor)
        
        # accumulate scalar performance metrics at each epoch to track and plot later
        train_losses.append(epoch_loss)
        valid_losses.append(valid_loss)

        if epoch % 3 == 0:
            save_checkpoint(optimizer, model, epoch, model_path/'v2.1-{epoch:03d}.pkl')
        
        # keep track of each epoch's progress
        print(f'\nEpoch {epoch} --- loss:{epoch_loss:.3f}, Epoch f-score:{epoch_acc:.2f}%, Validation loss:{valid_loss:.3f}, Validation f-score:{valid_acc:.2f}%')

In [52]:
# choose number of epochs higher than reasonable so we can manually stop training 
num_epochs = 500

# train it!
train(optimizer, model, num_epochs, X_train, y_train, X_valid, y_valid)

Epoch 0: iteration 192/193
Epoch 0 --- loss:1.828, Epoch f-score:0.39%, Validation loss:1.452, Validation f-score:0.31%
Epoch 1: iteration 192/193
Epoch 1 --- loss:1.410, Epoch f-score:0.40%, Validation loss:1.331, Validation f-score:0.33%
Epoch 2: iteration 192/193
Epoch 2 --- loss:1.307, Epoch f-score:0.41%, Validation loss:1.341, Validation f-score:0.38%
Epoch 3: iteration 192/193
Epoch 3 --- loss:1.265, Epoch f-score:0.42%, Validation loss:1.279, Validation f-score:0.36%
Epoch 4: iteration 192/193
Epoch 4 --- loss:1.230, Epoch f-score:0.43%, Validation loss:1.400, Validation f-score:0.35%
Epoch 5: iteration 192/193
Epoch 5 --- loss:1.199, Epoch f-score:0.45%, Validation loss:1.259, Validation f-score:0.38%
Epoch 6: iteration 192/193
Epoch 6 --- loss:1.188, Epoch f-score:0.45%, Validation loss:1.413, Validation f-score:0.34%
Epoch 7: iteration 192/193
Epoch 7 --- loss:1.183, Epoch f-score:0.45%, Validation loss:1.387, Validation f-score:0.35%
Epoch 8: iteration 192/193
Epoch 8 --- l

In [53]:
root_path = Path('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser'); root_path.ls()

[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v0'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v2.1'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/ser-data-v2-1.tar.gz')]

In [59]:
root_path.parent.parent.ls()

[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/NLU-dl.ipynb'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/dl4slp-playground.ipynb'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/dl4slp-audio.ipynb'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/result-prediction'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/nlu'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser')]

In [61]:
ser_path = root_path.parent.parent/'ser'; ser_path.exists()

True

In [62]:
!cp -R '/content/models/checkpoints'  {ser_path}

In [64]:
!ls {ser_path}

checkpoints


---

# Move back to origin framework

In [194]:
# opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)
loss_func = LabelSmoothingCrossEntropy()
lr = 1e-2
pct_start = 0.5
phases = create_phases(pct_start)
sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_mom = combine_scheds(phases, cos_1cycle_anneal(0.95,0.85, 0.95))
cbscheds = [ParamScheduler('lr', sched_lr), 
            ParamScheduler('mom', sched_mom)]

In [21]:
!pip install -qq torchmetrics
from torchmetrics import F1
f1 = F1(num_classes=4)

     |████████████████████████████████| 332 kB 5.1 MB/s 


In [188]:
import inspect
def show_source(m): print(inspect.getsource(m))

In [191]:
show_source(accuracy)

def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()



In [ ]:
class Learner():
    def __init__(self, model, data, loss_func, opt_func=sgd_opt, lr=1e-2, splitter=param_getter,
                 cbs=None, cb_funcs=None):
        self.model,self.data,self.loss_func,self.opt_func,self.lr,self.splitter = model,data,loss_func,opt_func,lr,splitter
        self.in_train,self.logger,self.opt = False,print,None

        # NB: Things marked "NEW" are covered in lesson 12
        # NEW: avoid need for set_runner
        self.cbs = []
        self.add_cb(TrainEvalCallback())
        self.add_cbs(cbs)
        self.add_cbs(cbf() for cbf in listify(cb_funcs))

    def add_cbs(self, cbs):
        for cb in listify(cbs): self.add_cb(cb)

    def add_cb(self, cb):
        cb.set_runner(self)
        setattr(self, cb.name, cb)
        self.cbs.append(cb)

    def remove_cbs(self, cbs):
        for cb in listify(cbs): self.cbs.remove(cb)

    def one_batch(self, i, xb, yb):
        try:
            self.iter = i
            self.xb,self.yb = xb,yb;                        self('begin_batch')
            self.pred = self.model(self.xb);                self('after_pred')
            self.loss = self.loss_func(self.pred, self.yb); self('after_loss')
            if not self.in_train: return
            self.loss.backward();                           self('after_backward')
            self.opt.step();                                self('after_step')
            self.opt.zero_grad()
        except CancelBatchException:                        self('after_cancel_batch')
        finally:                                            self('after_batch')

    def all_batches(self):
        self.iters = len(self.dl)
        try:
            for i,(xb,yb) in enumerate(self.dl): self.one_batch(i, xb, yb)
        except CancelEpochException: self('after_cancel_epoch')

    def do_begin_fit(self, epochs):
        self.epochs,self.loss = epochs,tensor(0.)
        self('begin_fit')

    def do_begin_epoch(self, epoch):
        self.epoch,self.dl = epoch,self.data.train_dl
        return self('begin_epoch')

    def fit(self, epochs, cbs=None, reset_opt=False):
        # NEW: pass callbacks to fit() and have them removed when done
        self.add_cbs(cbs)
        # NEW: create optimizer on fit(), optionally replacing existing
        if reset_opt or not self.opt: self.opt = self.opt_func(self.splitter(self.model), lr=self.lr)

        try:
            self.do_begin_fit(epochs)
            for epoch in range(epochs):
                self.do_begin_epoch(epoch)
                if not self('begin_epoch'): self.all_batches()

                with torch.no_grad():
                    self.dl = self.data.valid_dl
                    if not self('begin_validate'): self.all_batches()
                self('after_epoch')

        except CancelTrainException: self('after_cancel_train')
        finally:
            self('after_fit')
            self.remove_cbs(cbs)

    ALL_CBS = {'begin_batch', 'after_pred', 'after_loss', 'after_backward', 'after_step',
        'after_cancel_batch', 'after_batch', 'after_cancel_epoch', 'begin_fit',
        'begin_epoch', 'begin_epoch', 'begin_validate', 'after_epoch',
        'after_cancel_train', 'after_fit'}

    def __call__(self, cb_name):
        res = False
        assert cb_name in self.ALL_CBS
        for cb in sorted(self.cbs, key=lambda x: x._order): res = cb(cb_name) and res
        return res


In [ ]:
def learner(arch, data, loss_func, opt_func, c_in=None, c_out=None,
                lr=1e-2, cuda=True, norm=None, progress=True, mixup=0, xtra_cb=None, **kwargs):
    cbfs = [partial(AvgStatsCallback,accuracy)]+listify(xtra_cb)
    if progress: cbfs.append(ProgressCallback)
    if cuda:     cbfs.append(CudaCallback)
    if norm:     cbfs.append(partial(BatchTransformXCallback, norm))
    if mixup:    cbfs.append(partial(MixUp, mixup))
    arch_args = {}
    if not c_in : c_in  = data.c_in
    if not c_out: c_out = data.c_out
    if c_in:  arch_args['c_in' ]=c_in
    if c_out: arch_args['c_out']=c_out
    return Learner(arch, data, loss_func, opt_func=opt_func, lr=lr, cb_funcs=cbfs, **kwargs)


In [ ]:
model = Model(c_out).cuda()

In [ ]:
def sgd_opt(xtra_step=None, **kwargs):
    return partial(Optimizer, steppers=[weight_decay]+listify(xtra_step),
                   stats=[AverageGrad(dampening=True), AverageSqrGrad(), StepCount()], **kwargs)

In [ ]:
opt_func = sgd_opt(lr=0.01, weight_decay=1e-3, momentum=0.8)

In [ ]:
learn = learner(model, data, loss_func, opt_func=opt_func)

In [ ]:
learn.fit(300, cbs=cbscheds)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.520974,0.268680,1.658805,0.289404,00:10
1,2.498805,0.259936,1.663060,0.288079,00:11
2,2.516170,0.267091,1.656659,0.288742,00:11
3,2.515422,0.264388,1.661606,0.288742,00:10
4,2.506654,0.275676,1.659923,0.288742,00:10
5,2.520093,0.267568,1.663013,0.288742,00:10
6,2.508234,0.262957,1.662208,0.288742,00:10
7,2.511315,0.261685,1.652186,0.289404,00:10
8,2.508366,0.269316,1.662082,0.288079,00:10
9,2.508068,0.268362,1.662718,0.288079,00:10


KeyboardInterrupt: ignored

## Save model for inference

In [ ]:
%cd /content/

/content


In [ ]:
model_path = Path('/content/model/v9'); model_path.mkdir(parents = True, exist_ok=True)

In [ ]:
torch.save(learn.model.state_dict(), model_path/'ser-v9-transformer-cnn-mh-2.pt')

In [ ]:
!tar -czf model-v5-8.tar.gz model

# References

- [Pytorch Transformer Encoder](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html)
- [fastai course-v3 part2, audio](https://github.com/fastai/course-v3/blob/master/nbs/dl2/audio.ipynb)
- [Transformer-CNN-emotion recognition](https://github.com/IliaZenkov/transformer-cnn-emotion-recognition/blob/main/notebooks/Parallel_is_All_You_Want.ipynb)
- 
